In [25]:
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import re


In [26]:
with open('data/with_titles_and_abstract.json', 'r') as fin:
    data = json.load(fin)

In [27]:
df = pd.DataFrame(data)

In [28]:
columns_to_drop = [col for col in df.columns if col not in ['title', 'abstract']]

df = df.drop(columns=columns_to_drop, axis=1)

In [29]:
df = df.sample(frac=0.01)

In [30]:
df

,title,abstract
1483493,GRASP: Graph Alignment through Spectral Signat...,What is the best way to match the nodes of t...
1839264,Chandra observations of five X-ray transient g...,We report on exploratory Chandra observation...
1532927,Distributed Mission Planning of Complex Tasks ...,"In this paper, we propose a distributed mult..."
1514342,Second order estimates for augment Hessian equ...,The author extends previous results to gener...
719031,A machine-learning approach to measuring the e...,Recent observations of galaxies at $z \gtrsi...
...,...,...
405968,Hydrogen atom as a quantum-classical hybrid sy...,Hydrogen atom is studied as a quantum-classi...
1813772,Serendipitous 2MASS Discoveries Near the Galac...,We present the basic properties of three obj...
1712006,Multiple View Performers for Shape Completion,We propose the Multiple View Performer (MVP)...
1600406,A Psychoacoustic Quality Criterion for Path-Tr...,"In developing virtual acoustic environments,..."


In [31]:

train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [32]:
import random

def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    abstracts = df['abstract'].tolist()
    titles = df['title'].tolist()
    datapoints = [titles[i]+': '+abstracts[i] for i in range(len(titles))]
    for item in datapoints:
        if random.random() < 1:
            item = str(item).strip()
            item = re.sub(r"\s", " ", item)
            bos_token = '<|endoftext|>'
            eos_token = '<|endoftext|>'
            data += bos_token + ' ' + item + ' ' + eos_token + '\n'
        
    f.write(data)


In [33]:
build_dataset(df_train, 'data/gpt2_train.txt')
build_dataset(df_valid, 'data/gpt2_valid.txt')
build_dataset(df_test, 'data/gpt2_test.txt')

In [34]:
df.columns

Index(['title', 'abstract'], dtype='object')

In [35]:
df_train.to_json('data/train_small_best.json')
df_test.to_json('data/test_small_best.json')
df_valid.to_json('data/valid_small_best.json')

In [36]:
def build_json_dataset(df, path):
    with open(path, 'w') as fout:
        for i in df['title'].keys():
            item = {'title':df['title'][i], 'abstract':df['abstract'][i]}
            fout.write(json.dumps(item)+'\n')

In [37]:
build_json_dataset(df_train, 'data/train_small_best.json')
build_json_dataset(df_valid, 'data/valid_small_best.json')
build_json_dataset(df_test, 'data/test_small_best.json')

In [38]:
json.dumps({'title':df_train['title'][list(df_train['title'].keys())[0]], 'abstract':df_train['abstract'][list(df_train['title'].keys())[0]]})

'{"title": "Bulges", "abstract": "  We model the evolution of the galactic bulge and of the bulges of a selected\\nsample of external spiral galaxies, via the multiphase multizone evolution\\nmodel. We address a few questions concerning the role of the bulges within\\ngalactic evolution schemes and the properties of bulge stellar populations. We\\nprovide solutions to the problems of chemical abundances and spectral indices,\\nthe two main observational constraints to bulge structure.\\n"}'

In [ ]:
df_train

In [ ]:
d = df_train.to_dict()

In [ ]:
list(d['title'].keys()) == list(d['abstract'].keys())

In [ ]:
d['abstract']

In [ ]:
from datasets import load_dataset

train_set = load_dataset('json', data_files='data/train_small.json')

In [ ]:
train_set